In [1]:
# This function helps us figure out how to stitch two DNA reads (a and b) together.
def overlap(a, b, min_length=3):
    """ 
    Return the length of the longest suffix of 'a' that matches a prefix of 'b'.
    
    """
    
    # start searching for an overlap at the beginning of read 'a'.
    start = 0
    
    # This loop will keep running until we've either found the best possible overlap or we've run out of places to look in read 'a'.
    while True:
        # taking a small "seed" from the start of read 'b' and trying to find it in read 'a'.
        start = a.find(b[:min_length], start)
        
        if start == -1:  #  it means the seed wasn't found in the rest of string 'a'.
            # no overlap is possible. 
            return 0
            
        # If we found the seed, we need to verify it's a proper suffix/prefix match.
        # We check if the end of 'a' matches the start of 'b'.
        if b.startswith(a[start:]):
            return len(a) - start  # gives length of the overlap 
            
        # If it wasn't a suffix/prefix match we need to keep searching. 
        # We increment 'start' by 1 to look for the next possible occurrence of the seed.
        start += 1


In [2]:
import itertools # We need this to get all possible orderings of our reads.

def scs(ss):
    """ Finds the shortest common superstring for a list of strings. """
    shortest_sup = None # This will hold the best superstring we've found so far.
    
    # [brute-force] : We're going to try every possible permutation of the input strings.
    for ssperm in itertools.permutations(ss):
        # Start building a potential superstring with the first string in the current order.
        sup = ssperm[0]

        for i in range(len(ss)-1):
            # Figure out how much the current string and the next one overlap with the overlap function.
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            
            # Add the part of the next string that doesn't overlap, we stitch them together.
            sup += ssperm[i+1][olen:]
            
        # check if it's the best superstring yet.
        if shortest_sup is None or len(sup) < len(shortest_sup):
            # If it's our first one, or shorter than our previous best, we have a new winner.
            shortest_sup = sup
   
    return shortest_sup


In [3]:
def pick_maximal_overlap(reads, k):
    """
    Finds the best overlap between any two reads in a list.
    The overlap has to be at least 'k' bases long.
    """
    # These will hold the two reads that have the best overlap we've found.
    reada, readb = None, None
    # And this will store how long that best overlap is.
    best_olen = 0
    
    # Check every possible pair of reads.
    for a, b in itertools.permutations(reads, 2):
        # Calculate the overlap in the pair.
        olen = overlap(a, b, min_length=k)
        
        # If this overlap is the best one we've seen so far...
        if olen > best_olen:
            # ...then we'll remember this pair of reads and the overlap length.
            reada, readb = a, b
            best_olen = olen
            
    # After checking all the pairs, return the winning pair and their overlap length.
    return reada, readb, best_olen


In [8]:
def greedy_scs(reads, k):
    """
    This is a "greedy" way to find the shortest common superstring.
    It repeatedly finds the best overlap and merges the two reads.
    """
    
    # First, find the single best overlap in the entire set of reads.
    read_a, read_b, olen = pick_maximal_overlap(reads, k)
    
    # Keep merging as long as we can find overlaps of at least length k.
    while olen > 0:
        # Once we have the best pair, remove them from our list of reads.
        reads.remove(read_a)
        reads.remove(read_b)
        
        # Append the non-overlapping part of read_b to read_a.
        merged_read = read_a + read_b[olen:]
        reads.append(merged_read)
        
        # Now, with our newly merged read in the list, find the *next* best overlap.
        read_a, read_b, olen = pick_maximal_overlap(reads, k)
        
    # When no more overlaps can be found, join everything that's left into one final sequence.
    # *If it worked perfectly, there should only be one string left*
    return ''.join(reads)

In [12]:
greedy_scs(['ACGTGCGC', 'ATGCGCGAT', 'AGGCTGGCGC'], 2)

'ACGTGCGCATGCGCGATAGGCTGGCGC'

In [13]:
scs(['ACGTGCGC', 'ATGCGCGAT', 'AGGCTGGCGC'])

'ACGTGCGCATGCGCGATAGGCTGGCGC'